<img src="pexels-afta-putta-gunawan-683039.jpg" alt="coffee shop">

# Coffee Meet Data

Our client, a fictional coffee chain in New York, has asked us to take a look at their records. They want insight into any important factors that may help improve their business. They've provided a selection of sales, product, customer, and other records.

We'll use a combination of **Pandas** and **SQL** to pull out any relevant information. Then we'll chart this information using popular interactive visualization library **Plotly Express**.

Click Here for fully Interactive Charts and output*: [nbviewer](https://nbviewer.org/github/nembdev/python_portfolio/blob/main/Data%20Analysis%20Projects/Coffee/nbviewer%20display%20code/Coffee%20Shop%20Analysis-Display.ipynb) 

## Dataset

There are nine tables of sample retail data provided:

* Sales Receipts
* Pastry Inventory
* Sales Targets
* Customer
* Dates
* Product
* Sales Outlet
* Staff
* Generation

Source: [IBM/Kaggle](https://www.kaggle.com/datasets/ylchang/coffee-shop-sample-data-1113)


## Potential Insights


1. [Busiest time of day](#time)
2. [Staff contribution ranking](#staff_rank)
3. [Customer Analysis](#customer)
4. Best selling products
5. Customer generational makeup
6. Online vs Instore
7. KPI Example



# Data Dictionary - Customers

|Column Name| Description|
|-----------|-----------|
|**customer_id**|A unique id assigned upon registration to each customer|
|**home_store**|The store a customer was registered at or has set as their main location|
|**customer_first-name**|Customers first name and last name|
|**customer_email**|Customers email used for registration|
|**customer_since**|How long a customer has been registered or when they first started shopping.|
|**loyalty_card_number**|A unique id used for a loyalty rewards program|
|**birthdate**|Customers Date of Birth - YYYY-MM-DD|
|**gender**|Customers registered gender|
|**birth_year**|Customers Year of Birth |

# Data Dictionary - Sales

|Column Name| Description|
|-----------|-----------|
|**transaction_id**|A unique id assigned to each purchase.|
|**transaction_date**|Calendar date when a transaction was made YYYY-MM-DD|
|**transaction_time**|Time when a transaction was made HH-MM-SS|
|**sales_outlet_id**|A unique used to identify which store handled the sale|
|**staff_id**|Unique id assigned to each staff member. Based on which staff member processed the transaction|
|**customer_id**|A unique id assigned upon registration to each customer|
|**instore_yn**|Was this purchase made in store or online|
|**order**|Unspecified|
|**line_item_id**|Unspecified|
|**product_id**|A unique id assigned to each product|
|**quantity**|The amount of each product purchased|
|**line_item_amount**|Unspecified|
|**unit_price**|Unit price for the product specified|
|**promo_item_yn**|Was the product a part of a promotional campaign|


## Library Imports

In [ ]:
# data
import pandas as pd

# offline charting
import plotly.io as pio

pio.renderers.default = "notebook"
pio.kaleido.scope.default_format = "svg"

# visuals
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# date and time
import datetime as dt
from datetime import datetime

# sql
import sqlite3

# system
from pathlib import Path

In [ ]:
%%HTML
<style>
td {
  font-size: 18px
}
</style>

## Database Connection

In a real world scenario your dataset isn't alway contained in a handy cleaned csv file, but in a production database.

Here we create and populate a sqlite3 database in order to simulate real world conditions.

We use pandas to convert our sql queries into a dataframe for analysis.

In [ ]:
# create our initial db file
Path("coffee.db").touch()

# create a db connection
connection = sqlite3.connect("coffee.db")

# cursor - database iterator
c = connection.cursor()

# create a sample table to verify
# c.execute('''CREATE TABLE sample_table (u_id int, email text)''')

# load the data into a dataframe
customers = pd.read_csv("customer.csv")

# write df to a sqlite table
customers.to_sql(
    "customers", connection, if_exists="append", index=False
)  # 2246 results

# data for our second table/df
sales = pd.read_csv("2019_04_sales_reciepts.csv")

# second table
sales.to_sql("sales", connection, if_exists="append", index=False)  # 49894 results

# verify
# fetchall returns an array of tuples
# c.execute('''SELECT *  FROM customers''').fetchall()

# verify second table
# c.execute('''SELECT *  FROM sales''').fetchall()

# join our two tables
c.execute(
    """SELECT * FROM sales s LEFT JOIN customers c ON c.customer_id = s.customer_id"""
)
# c.fetchall()

## Query to Dataframe conversion

Once we've pulled the data we need from our database, it needs to be converted into dataframe for ease of analysis.

The customer_id column was duplicated after our conversion, so we can safely drop it using a ~ (bitwise negation operator) and a call to col.duplicated().

In [ ]:
# import our joined sql tables into a dataframe
coffee = pd.read_sql(
    """SELECT * FROM sales s LEFT JOIN customers c ON c.customer_id = s.customer_id""",
    connection,
)

# delete duplicate column
coffee = coffee.loc[:, ~coffee.columns.duplicated()]

In [ ]:
coffee.head(5)

In [ ]:
coffee.tail(5)

In [ ]:
coffee.info()

In [ ]:
coffee.columns

# Data Cleaning - Missing Values

Our info method shows us that each of these columns contain missing values"

* home_store
* customer_first-name 
* customer_email
* customer_since
* loyalty_card_number
* birthdate 
* gender
* birth_year           

This was caused when we joined our sales and customer tables. 

For now we'll fill any missing values with a 0. 

This lets us convert as we need without running into numerical errors.

In [ ]:
coffee = coffee.fillna(0)

## Data Cleaning - Column Rename

### Customer -

customer_id	home_store	customer_first-name	customer_email	customer_since	loyalty_card_number	birthdate	gender	birth_year

### Sales

transaction_id	transaction_date	transaction_time	sales_outlet_id	staff_id	customer_id	instore_yn	order	line_item_id	product_id	quantity	line_item_amount	unit_price	promo_item_yn

### Changes 
Column names were shortened or adjusted for clarity.

|Old Name|New Name|
---------------|---------------
|sales_outlet_id|shop_id|
|instore_yn|instore_purchase|
|promo_item_yn'|promo_item|
|customer_first-name|name|
|customer_email|email|
|loyalty_card_number|loyalty_id

In [ ]:
coffee.rename(
    columns={
        "sales_outlet_id": "shop_id",
        "instore_yn": "instore_purchase",
        "promo_item_yn": "promo_item",
        "customer_first-name": "customer_name",
        "customer_email": "email",
        "loyalty_card_number": "loyalty_id",
    },
    inplace=True,
)
coffee.columns

## Data Cleaning - Y/N to Bool

The instore_purchase column signifies whether a purchase was made **in a store or online**. The value is stored as string of Y for yes or N for no. We will convert this into a **true/false** boolean for ease of analysis.

In [ ]:
coffee.loc[coffee["instore_purchase"] == "Y", "instore_purchase"] = True
coffee.loc[coffee["instore_purchase"] == "N", "instore_purchase"] = False
coffee["instore_purchase"] = coffee["instore_purchase"].astype(bool)

## Data Cleaning - Datetime Conversion

We'll convert our time and date columns to datetime objects. This will help us manipulate and plot our time data correctly. Any missing birth_years are set to 1900, being the earliest year recognized by an excel file. This is simple substitution that can be filtered out later.

In [ ]:
coffee["transaction_date"] = pd.to_datetime(coffee["transaction_date"])
coffee["transaction_time"] = pd.to_datetime(coffee["transaction_time"])
coffee["birthdate"] = pd.to_datetime(coffee["birthdate"])

# special formating for year
coffee["birth_year"] = coffee["birth_year"].astype(int)
coffee.loc[
    coffee["birth_year"] == 0, "birth_year"
] = 1900  # placeholder for Missing Values
coffee["birth_year"] = pd.to_datetime(coffee["birth_year"], format="%Y")

# verify
coffee[["transaction_date", "transaction_time", "birthdate", "birth_year"]].head(5)

In [ ]:
# Misc Cleaning - converting home store datatype
coffee["home_store"] = coffee["home_store"].astype(int)

## Information Gathering

1. How many stores are we working with?
2. When are the stores open/taking orders?

## Number of Locations

Our dataset contains **3 stores** with ids of 3, 5, and 8.

Each store also takes online orders.

We'll separate our coffee data based on store id and online vs inperson.

In [ ]:
# Stores
coffee["shop_id"].unique()

In [ ]:
coffee_inperson = coffee[coffee["instore_purchase"] == True]
coffee_online = coffee[coffee["instore_purchase"] == False]


store3 = coffee[coffee["shop_id"] == 3]
store3_inperson = coffee.loc[
    (coffee["shop_id"] == 3) & (coffee["instore_purchase"] == True)
]
store3_online = coffee.loc[
    (coffee["shop_id"] == 3) & (coffee["instore_purchase"] == False)
]

store5 = coffee[coffee["shop_id"] == 5]
store5_inperson = coffee.loc[
    (coffee["shop_id"] == 5) & (coffee["instore_purchase"] == True)
]
store5_online = coffee.loc[
    (coffee["shop_id"] == 5) & (coffee["instore_purchase"] == False)
]

store8 = coffee[coffee["shop_id"] == 8]
store8_inperson = coffee.loc[
    (coffee["shop_id"] == 8) & (coffee["instore_purchase"] == True)
]
store8_online = coffee.loc[
    (coffee["shop_id"] == 8) & (coffee["instore_purchase"] == False)
]

## Store Hours

Based off our transactions times, we can deduce that the coffee shops **open around 6am and close around 8pm**. 

**Store 5** however opens as early as **1am**.

In [ ]:
print("Store 3")
print("Earliest  Order:", store3["transaction_time"].dt.hour.min())
print("Latest Order:", store3["transaction_time"].dt.hour.max(), "\n")
print("Store 5")
print("Earliest  Order:", store5["transaction_time"].dt.hour.min())
print("Latest Order:", store5["transaction_time"].dt.hour.max(), "\n")
print("Store 8")
print("Earliest  Order:", store8["transaction_time"].dt.hour.min())
print("Latest Order:", store8["transaction_time"].dt.hour.max())

## Busiest Time of Day
<a id='time'></a>

Knowing exactly when most of your customers/orders will arrive can help drastically increase profits, cut down on waste, and help you pinpoint when to increase staff availability.

Using our transaction data we can identify spikes in customer shopping down to the exact hour and day.

In [ ]:
# inperson transaction_date
fig = make_subplots(
    rows=2,
    cols=2,
    start_cell="top-left",
    subplot_titles=("Store 3", "Store 5", "Store 8", "All Stores"),
)

fig.add_trace(
    go.Histogram(x=store3_inperson["transaction_date"].dt.day_name()), row=1, col=1
)

fig.add_trace(
    go.Histogram(x=store5_inperson["transaction_date"].dt.day_name()), row=1, col=2
)

fig.add_trace(
    go.Histogram(x=store8_inperson["transaction_date"].dt.day_name()), row=2, col=1
)

fig.add_trace(
    go.Histogram(x=coffee_inperson["transaction_date"].dt.day_name()), row=2, col=2
)

fig.update_layout(title_text="Store In-Person Daily Volume", title_x=0.5)

fig.update_layout(
    yaxis=dict(range=[0, 1800]),
    yaxis2=dict(range=[0, 1800]),
    yaxis3=dict(range=[0, 1800]),
    yaxis4=dict(range=[0, 4500]),
)

fig.show("notebook")  # interactive nbviewer
# fig.show(renderer="svg", width=1000, height=800) # svg static for github

In [ ]:
# inperson transaction_date
fig = make_subplots(
    rows=2,
    cols=2,
    start_cell="top-left",
    subplot_titles=("Store 3", "Store 5", "Store 8", "All Stores"),
)

fig.add_trace(
    go.Histogram(x=store3_online["transaction_date"].dt.day_name()), row=1, col=1
)

fig.add_trace(
    go.Histogram(x=store5_online["transaction_date"].dt.day_name()), row=1, col=2
)

fig.add_trace(
    go.Histogram(x=store8_online["transaction_date"].dt.day_name()), row=2, col=1
)

fig.add_trace(
    go.Histogram(x=coffee_online["transaction_date"].dt.day_name()), row=2, col=2
)

fig.update_layout(title_text="Store Online Daily Volume", title_x=0.5)

fig.update_layout(
    yaxis=dict(range=[0, 1800]),
    yaxis2=dict(range=[0, 1800]),
    yaxis3=dict(range=[0, 1800]),
    yaxis4=dict(range=[0, 4500]),
)

fig.show("notebook")  # interactive nbviewer
# fig.show(renderer="svg", width=1000, height=800) # svg static for github

In [ ]:
# inperson transaction_time
fig = make_subplots(
    rows=2,
    cols=2,
    start_cell="top-left",
    subplot_titles=("Store 3", "Store 5", "Store 8", "All Stores"),
)


fig.add_trace(go.Histogram(x=store3_inperson["transaction_time"].dt.hour), row=1, col=1)

fig.add_trace(go.Histogram(x=store5_inperson["transaction_time"].dt.hour), row=1, col=2)

fig.add_trace(go.Histogram(x=store8_inperson["transaction_time"].dt.hour), row=2, col=1)

fig.add_trace(go.Histogram(x=coffee_inperson["transaction_time"].dt.hour), row=2, col=2)

fig.update_layout(
    title_text="Store In-Person Hourly Volume",
    title_x=0.5,
    xaxis={"dtick": 1},
    xaxis2={"dtick": 1},
    xaxis3={"dtick": 1},
)

fig.update_layout(
    yaxis=dict(range=[0, 1200]),
    yaxis2=dict(range=[0, 1200]),
    yaxis3=dict(range=[0, 1200]),
    yaxis4=dict(range=[0, 3500]),
)


fig.show("notebook")  # interactive nbviewer
# fig.show(renderer="svg", width=1000, height=800) # svg static for github

In [ ]:
# online transaction_time
fig = make_subplots(
    rows=2,
    cols=2,
    start_cell="top-left",
    subplot_titles=("Store 3", "Store 5", "Store 8", "All Stores"),
)

fig.add_trace(go.Histogram(x=store3_online["transaction_time"].dt.hour), row=1, col=1)

fig.add_trace(go.Histogram(x=store5_online["transaction_time"].dt.hour), row=1, col=2)

fig.add_trace(go.Histogram(x=store8_online["transaction_time"].dt.hour), row=2, col=1)

fig.add_trace(go.Histogram(x=coffee_online["transaction_time"].dt.hour), row=2, col=2)

fig.update_layout(
    title_text="Store Online Hourly Volume",
    title_x=0.5,
    xaxis={"dtick": 1},
    xaxis2={"dtick": 1},
    xaxis3={"dtick": 1},
)

fig.update_layout(
    yaxis=dict(range=[0, 1200]),
    yaxis2=dict(range=[0, 1200]),
    yaxis3=dict(range=[0, 1200]),
    yaxis4=dict(range=[0, 3500]),
)


fig.show("notebook")  # interactive nbviewer
# fig.show(renderer="svg", width=1000, height=800) # svg static for github

## Single Store Comparison - Store 3

In [ ]:
# store 3 all comparison
fig = make_subplots(
    rows=2,
    cols=2,
    start_cell="top-left",
    subplot_titles=(
        "In-Person Hourly",
        "In-Person Daily Volume",
        "Online Hourly Volume",
        "Online Daily Volume",
    ),
)


fig.add_trace(go.Histogram(x=store3_inperson["transaction_time"].dt.hour), row=1, col=1)

fig.add_trace(
    go.Histogram(x=store3_inperson["transaction_date"].dt.day_name()), row=1, col=2
)

fig.add_trace(go.Histogram(x=store3_online["transaction_time"].dt.hour), row=2, col=1)

fig.add_trace(
    go.Histogram(x=store3_online["transaction_date"].dt.day_name()), row=2, col=2
)

fig.update_layout(
    title_text="Store 3", title_x=0.5, xaxis={"dtick": 1}, xaxis3={"dtick": 1}
)

fig.update_layout(yaxis=dict(range=[0, 1200]), yaxis3=dict(range=[0, 1200]))


fig.show("notebook")  # interactive nbviewer
# fig.show(renderer="svg", width=1000, height=800) # svg static for github

## Staff contribution ranking
<a id='staff_rank'></a>

In [ ]:
# load in our staff data
staff = pd.read_csv("staff.csv")
# drop two misformed empty columns
staff.drop(staff.columns[[6, 7]], axis=1, inplace=True)

# combine our staff name columns into one
staff["staff_name"] = staff["first_name"] + " " + staff["last_name"]
# drop redundant naming columns
staff.drop(staff.columns[[1, 2]], axis=1, inplace=True)

# merge
coffee = pd.merge(left=coffee, right=staff)
coffee.columns

In [ ]:
# get all staff ids
coffee["staff_id"].unique()

## Finding Total Sales 

In [ ]:
total_sales = []
names = []
pie = {}

for staff_id in coffee["staff_id"].unique():
    """
    Takes an array/series of staff id's and calculates the total amount of sales by that staff memeber
    uses quantity x unit_price
    """
    # get staff id
    current_staff = coffee["staff_id"] == staff_id

    # get staff name & store in list
    name = coffee[current_staff]["staff_name"]
    names.append(name.iloc[0])
    name = name.iloc[0]

    # find total sales by staff
    cs_sales = coffee[current_staff]["unit_price"] * coffee[current_staff]["quantity"]
    cs_sales_total = cs_sales.sum()

    # calculate sum & store in list
    total_sales.append(cs_sales_total)
    pie[name] = cs_sales_total


# convert into a dataframe
total_sales_by_staff = pd.DataFrame({"staff_name": names, "total_sales": total_sales})
# verify
# total_sales_by_staff

In [ ]:
# setup up our chart
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Bar(
        x=total_sales_by_staff["staff_name"],
        y=total_sales_by_staff["total_sales"],
        marker={"color": total_sales_by_staff["total_sales"], "colorscale": "agsunset"},
    )
)

fig.update_layout(
    title_text="Total Sales Per Staff Member",
    title_x=0.5,
    barmode="stack",
    xaxis={"categoryorder": "total descending"},
)

fig.show("notebook")  # interactive nbviewer
# fig.show(renderer="svg", width=1000, height=800) # svg static for github

In [ ]:
fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Bar(
        x=total_sales_by_staff["staff_name"][0:5],
        y=total_sales_by_staff["total_sales"][0:5],
        marker={
            "color": total_sales_by_staff["total_sales"][0:5],
            "colorscale": "agsunset",
        },
    )
)

fig.update_layout(
    title_text="Top 5 Staff By Total Sales",
    title_x=0.5,
    barmode="stack",
    xaxis={"categoryorder": "total descending"},
)

fig.show("notebook")  # interactive nbviewer
# fig.show(renderer="svg", width=1000, height=800) # svg static for github

In [ ]:
last_sales = total_sales_by_staff.sort_values(by="total_sales")

fig = make_subplots(rows=1, cols=1)

fig.add_trace(
    go.Bar(
        x=last_sales["staff_name"][0:5],
        y=last_sales["total_sales"][0:5],
        marker={"color": last_sales["total_sales"][0:5], "colorscale": "agsunset"},
    )
)

fig.update_layout(
    title_text="Bottom 5 Staff By Total Sales",
    title_x=0.5,
    barmode="stack",
    xaxis={"categoryorder": "total descending"},
)

fig.show("notebook")  # interactive nbviewer
# fig.show(renderer="svg", width=1000, height=800) # svg static for github

In [ ]:
# fig = px.pie(df, values='pop', names='country', title='Population of European continent')

fig = make_subplots(rows=1, cols=1)

fig.add_trace(go.Pie(values=total_sales, labels=names))

fig.update_traces(hoverinfo="label+value", textinfo="percent")

fig.update_layout(
    title_text="Percentage of Sales By Staff Member",
    title_x=0.5,
    barmode="stack",
    xaxis={"categoryorder": "total descending"},
)

fig.show("notebook")  # interactive nbviewer

# Customer Analysis

1. Longest Registered Customer
2. Customers by Total Purchases
3. Customers by Age
